In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os

# Define the path to your ZIP file
zip_path = "/content/drive/MyDrive/BrainMRI.zip"
# Change extract_path to a directory, not the zip file itself
extract_path = "/content/drive/MyDrive/BrainImages"

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")

Dataset extracted successfully!


In [ ]:
import os
import shutil


base_path = "/content/drive/MyDrive/BrainImages"


for folder in ["train", "test"]:
    nested_path = os.path.join(base_path, folder, folder)
    correct_path = os.path.join(base_path, folder)

    if os.path.exists(nested_path):
        for category in os.listdir(nested_path):
            shutil.move(os.path.join(nested_path, category), correct_path)

        os.rmdir(nested_path)

print(" Dataset structure fixed!")


 Dataset structure fixed!


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
import re


In [3]:
import os
import shutil

dataset_path = "/content/drive/MyDrive/BrainImages"  # Change this to your dataset path

# Function to remove .ipynb_checkpoints folders
def remove_checkpoints(root_path):
    for dirpath, dirnames, filenames in os.walk(root_path):
        if ".ipynb_checkpoints" in dirnames:
            checkpoint_path = os.path.join(dirpath, ".ipynb_checkpoints")
            shutil.rmtree(checkpoint_path)
            print(f"Removed: {checkpoint_path}")

# Remove checkpoints in train, test, and val folders
remove_checkpoints(dataset_path)
print(" All .ipynb_checkpoints folders removed!")


 All .ipynb_checkpoints folders removed!


In [4]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((150, 150)),  # Resize images
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

# Load dataset
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/BrainImages/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/BrainImages/test', transform=transform)

# Create DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Get class names
class_names = train_dataset.classes
print(f'Classes: {class_names}')


Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [5]:
class BrainTumorCNN(nn.Module):
    def __init__(self, num_classes=4):  # Adjust based on dataset
        super(BrainTumorCNN, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 18 * 18, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [6]:
# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BrainTumorCNN(num_classes=len(class_names)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}')

# Train the model
train_model(model, train_loader, criterion, optimizer, epochs=10)


Epoch 1/10, Loss: 0.5161
Epoch 2/10, Loss: 0.2282
Epoch 3/10, Loss: 0.1245
Epoch 4/10, Loss: 0.0782
Epoch 5/10, Loss: 0.0551
Epoch 6/10, Loss: 0.0404
Epoch 7/10, Loss: 0.0304
Epoch 8/10, Loss: 0.0386
Epoch 9/10, Loss: 0.0264
Epoch 10/10, Loss: 0.0337


In [7]:
def evaluate_model(model, test_loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    print(f'Test Accuracy: {acc * 100:.2f}%')

# Evaluate model
evaluate_model(model, test_loader)


Test Accuracy: 90.88%


In [8]:
import PIL.Image

def predict_image(image_path, model, transform):
    image = PIL.Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    return class_names[predicted.item()]

# Test on an image
image_path = '/content/imageofbrain.jpg'
prediction = predict_image(image_path, model, transform)
print(f'Predicted Class: {prediction}')


Predicted Class: meningioma
